In [8]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [74]:
import warnings
warnings.filterwarnings('ignore')

In [71]:
#Creating the functiong that will get the data for us
def get_data(league):
    
    files = [file for file in os.listdir(f'./../raw_data/{league}')]
    data = pd.DataFrame()


    for file in files:
        df = pd.read_csv('./../raw_data/Turkey/'+file)
        data = pd.concat([data, df])
    return data

In [72]:
#Feature Engineering

In [73]:
def feature_engineering(data, b=20):
    '''
    This function creates all the columns that will be needed to create the analysis 
    and return the dataframe with all this changes
    
    b is the number of bins that we want to work with. Our start value for b will be 20.
    '''
        
    # total number of goals = goals from the home team + goals from visiting team
    data['nb_goals']=data['FTHG']+data['FTAG']

    # boolean: true or false regarding whether they were more than 2.5 goals
    data['over_2.5_goals']=data['nb_goals']>2.5

    # boolean: true or false regarding whether they were less than 2.5 goals
    data['under_2.5_goals']=data['nb_goals']<2.5

    # payout of betting on over 2.5 goals: we get 0 if we lose the bet, we get the AvgC if we win the bet (AvgC = market average of the odds)
    data['payout'] = data['over_2.5_goals']*data['AvgC>2.5']

    # payout of betting on under 2.5 goals: we get 0 if we lose the bet, we get the AvgC if we win the bet (AvgC = market average of the odds)
    data['payout_under_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

    #payout UNDER 2.5 for PINACLE specifically
    data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['PC<2.5']

    #payout OVER 2.5 for PINACLE specifically
    data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['PC>2.5']

    #payout UNDER 2.5 for 365 specifically
    data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365C<2.5']

    #payout OVER 2.5 for 365 specifically
    data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365C>2.5']
    
    #Implied Probability OVER 2.5 goals for overall market (AvgC)
    data['Implied Probability']=1/data['AvgC>2.5']*100

    #Implied Probability UNDER 2.5 goals for PINACLE
    data['Implied Probability <2.5 pinacle']=1/data['PC<2.5']*100

    #Implied Probability OVER 2.5 goals for PINACLE
    data['Implied Probability >2.5 pinacle']=1/data['PC>2.5']*100

    #Implied Probability UNDER 2.5 goals for 365
    data['Implied Probability <2.5 365']=1/data['B365C<2.5']*100

    #Implied Probability OVER 2.5 goals for 365
    data['Implied Probability >2.5 365']=1/data['B365C>2.5']*100
    
    # Binning the implied probabilities with bins of 10
    b=b
    bins = np.arange(0, 101, int(100/b))
    bins = bins.tolist()
    
    data['binned'] = pd.cut(data['Implied Probability'], bins)
    data['binned <2.5'] = pd.cut(data['Implied Probability <2.5'], bins)
    data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)
    data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)
    data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)
    data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)
    

    return data

In [65]:
def final_dfs(data, pinnacle=False, bet365=False):
    
    if pinnacle == True and bet365 == True:
        fdf = data[['Implied Probability','over_2.5_goals','binned','payout']]
        fdf_under = data[['Implied Probability <2.5','under_2.5_goals','binned <2.5','payout_under_2.5']]
        fdf_under_365 = data[['Implied Probability <2.5 365','under_2.5_goals','binned <2.5 365','payout_under_2.5_365']]
        fdf_under_pinacle = data[['Implied Probability <2.5 pinacle','under_2.5_goals','binned <2.5 pinacle','payout_under_2.5_pinacle']]
        fdf_over = data[['Implied Probability >2.5','over_2.5_goals','binned >2.5','payout_over_2.5']]
        fdf_over_pinacle = data[['Implied Probability >2.5 pinacle','over_2.5_goals','binned >2.5 pinacle','payout_over_2.5_pinacle']]
        fdf_over_365 = data[['Implied Probability >2.5 365','over_2.5_goals','binned >2.5 365','payout_over_2.5_365']]

        return 
    else:
        fdf = data[['Implied Probability','over_2.5_goals','binned','payout']]
        fdf_under = data[['Implied Probability <2.5','under_2.5_goals','binned <2.5','payout_under_2.5']]
        fdf_over = data[['Implied Probability >2.5','over_2.5_goals','binned >2.5','payout_over_2.5']]
        
        return 

In [66]:
bins = bins.tolist()

In [67]:
bins

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]